# LangChain应用集成讯飞星火认知大模型

## 集成指南

讯飞星火认知大模型是国内领先的大模型，拥有跨领域的知识和语言理解能力，能够完成问答对话和文学创作等任务。星火大模型持续从海量文本数据和大规模语法知识中学习进化，实现从提出问题、规划问题到解决问题的全流程闭环。

星火大模型通过API提供访问。您可以在[https://xinghuo.xfyun.cn/sparkapi](https://dub.sh/xinghuo)申请API访问。

讯飞星火的**认知大模型**聊天补全（chat completion）利用了不同于OpenAI的机制，它通过WebSocket提供访问。本指南向您演示如何通过FastAPI来提供OpenAI接口协议兼容的星火认知大模型接口，通过此方式，我们能够在**LangChain**应用中使用**星火认知大模型**。

[spark-api-gateway](https://github.com/sugarforever/spark-api-gateway)正是本示例所使用的API Gateway，通过该Gateway提供与OpenAI接口协议兼容的API。spark-api-gateway目前支持如下聊天补全API：

- POST /v1/chat/completions

### OpenAI Vision模型

OpenAI的Vision模型是通过聊天补全API提供服务的，请参考官方文档[OpenAI - Vision](https://platform.openai.com/docs/guides/vision)。

请求示例如下：

```shell
curl https://api.openai.com/v1/chat/completions \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -d '{
    "model": "gpt-4-vision-preview",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "What’s in this image?"
          },
          {
            "type": "image_url",
            "image_url": {
              "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
            }
          }
        ]
      }
    ],
    "max_tokens": 300
  }'
```

### Spark API网关

科大讯飞星火大模型的图片理解API独立于文本补全，其请求地址为：

```
wss://spark-api.cn-huabei-1.xf-yun.com/v2.1/image
```

为提供统一的访问体验，Spark API Gateway在 `/v1/chat/completions` 实现了对图片理解API的支持。开发者或用户可以通过OpenAI请求兼容的数据向星火大模型发送请求。示例请求如下：

```shell
curl --location 'https://sparkai-gateway.vercel.app/v1/chat/completions' \
--header 'Content-Type: application/json' \
--data '{
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://pbs.twimg.com/media/F_8fhwIWQAAMHe5?format=jpg&name=large"
                    }
                },
                {
                    "type": "text",
                    "text": "这张图里的标志是什么？"
                }
            ]
        }
    ],
    "model": "vision",
    "max_tokens": null,
    "stream": false,
    "n": 1,
    "temperature": 0.7,
    "version": "v2.1"
}'
```

In [6]:
! pip install langchain openai -q -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.8/220.8 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


## 示例1 - 简单的聊天补全

In [ ]:
from langchain.chat_models import ChatOpenAI

spark_chat = ChatOpenAI(openai_api_base="https://sparkai-gateway.vercel.app/v1", openai_api_key="sk-xxx")

In [ ]:
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
messages = [
    HumanMessage(content="大语言模型的理论基础是什么？"),
]

In [ ]:
spark_chat.invoke(messages)

AIMessage(content='大语言模型的理论基础是深度学习。大语言模型是深度学习的应用之一，尤其在自然语言处理(NLP)领域。这些模型的目标是理解和生成人类语言。为了实现这个目标，模型需要在大量文本数据上进行训练，以学习语言的各种模式和结构。')

## 示例2 - LLMChain使用星火认知大模型

In [ ]:

from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)
prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{human_input}")
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
from langchain.chains import LLMChain
chat_llm_chain = LLMChain(
    llm=spark_chat,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [ ]:
chat_llm_chain.predict(human_input="你是谁？")



> Entering new LLMChain chain...
Prompt after formatting:
Human: 你是谁？

> Finished chain.


'您好，我是科大讯飞研发的认知智能大模型，我的名字叫讯飞星火认知大模型。我可以和人类进行自然交流，解答问题，高效完成各领域认知智能需求。'

In [ ]:
chat_llm_chain.predict(human_input="请解释第一性原理")



> Entering new LLMChain chain...
Prompt after formatting:
Human: 你是谁？
AI: 您好，我是科大讯飞研发的认知智能大模型，我的名字叫讯飞星火认知大模型。我可以和人类进行自然交流，解答问题，高效完成各领域认知智能需求。
Human: 请解释第一性原理

> Finished chain.


'第一性原理是指，回归事物最基本的条件，将其拆分成各要素进行解构分析，从而找到实现目标最优路径的方法。  该原理源于古希腊哲学家亚里士多德提出的一个哲学观点：“每个系统中存在一个最基本的命题，它不能被违背或删除。” \n\n这个名词被炒得很火主要得益于“钢铁侠”埃隆·马斯克(特斯拉汽车CEO)。他曾在采访中提到自己特别推崇“第一性原理”思考法：“通过第一性原理，我把事情升华到最根本的真理，然后从最核心处开始推理......”'

In [ ]:
chat_llm_chain.predict(human_input="这位哲学家的最大成就是什么？")



> Entering new LLMChain chain...
Prompt after formatting:
Human: 你是谁？
AI: 您好，我是科大讯飞研发的认知智能大模型，我的名字叫讯飞星火认知大模型。我可以和人类进行自然交流，解答问题，高效完成各领域认知智能需求。
Human: 请解释第一性原理
AI: 第一性原理是指，回归事物最基本的条件，将其拆分成各要素进行解构分析，从而找到实现目标最优路径的方法。  该原理源于古希腊哲学家亚里士多德提出的一个哲学观点：“每个系统中存在一个最基本的命题，它不能被违背或删除。” 

这个名词被炒得很火主要得益于“钢铁侠”埃隆·马斯克(特斯拉汽车CEO)。他曾在采访中提到自己特别推崇“第一性原理”思考法：“通过第一性原理，我把事情升华到最根本的真理，然后从最核心处开始推理......”
Human: 这位哲学家的最大成就是什么？

> Finished chain.


'亚里士多德是古希腊哲学家，他在哲学上最大的贡献在于创立了形式逻辑这一重要分支学科。逻辑思维是亚里士多德在众多领域建树卓越的支柱，这种思维方式自始至终贯穿于他的研究、统计和思考之中。\n\n除此之外，他还是一位杰出的科学家，对物理学、生物学、植物学、动物学、解剖学、伦理学等领域都有很大的贡献。'

In [ ]:
chat_llm_chain.predict(human_input="他在心理学方面的贡献是什么？")



> Entering new LLMChain chain...
Prompt after formatting:
Human: 你是谁？
AI: 您好，我是科大讯飞研发的认知智能大模型，我的名字叫讯飞星火认知大模型。我可以和人类进行自然交流，解答问题，高效完成各领域认知智能需求。
Human: 请解释第一性原理
AI: 第一性原理是指，回归事物最基本的条件，将其拆分成各要素进行解构分析，从而找到实现目标最优路径的方法。  该原理源于古希腊哲学家亚里士多德提出的一个哲学观点：“每个系统中存在一个最基本的命题，它不能被违背或删除。” 

这个名词被炒得很火主要得益于“钢铁侠”埃隆·马斯克(特斯拉汽车CEO)。他曾在采访中提到自己特别推崇“第一性原理”思考法：“通过第一性原理，我把事情升华到最根本的真理，然后从最核心处开始推理......”
Human: 这位哲学家的最大成就是什么？
AI: 亚里士多德是古希腊哲学家，他在哲学上最大的贡献在于创立了形式逻辑这一重要分支学科。逻辑思维是亚里士多德在众多领域建树卓越的支柱，这种思维方式自始至终贯穿于他的研究、统计和思考之中。

除此之外，他还是一位杰出的科学家，对物理学、生物学、植物学、动物学、解剖学、伦理学等领域都有很大的贡献。
Human: 他在心理学方面的贡献是什么？

> Finished chain.


'亚里士多德在心理学方面的贡献是他对人类思维和行为的研究。他认为，人的思想和行为是由身体、情感和精神三部分组成的，这三部分相互作用，共同影响着人的行为。\n\n此外，亚里士多德还提出了一种被称为“幸福学”的学科，它研究的是人的幸福和快乐。'

In [ ]:
chat_llm_chain.predict(human_input="哪里能买到这本书？")



> Entering new LLMChain chain...
Prompt after formatting:
Human: 你是谁？
AI: 您好，我是科大讯飞研发的认知智能大模型，我的名字叫讯飞星火认知大模型。我可以和人类进行自然交流，解答问题，高效完成各领域认知智能需求。
Human: 请解释第一性原理
AI: 第一性原理是指，回归事物最基本的条件，将其拆分成各要素进行解构分析，从而找到实现目标最优路径的方法。  该原理源于古希腊哲学家亚里士多德提出的一个哲学观点：“每个系统中存在一个最基本的命题，它不能被违背或删除。” 

这个名词被炒得很火主要得益于“钢铁侠”埃隆·马斯克(特斯拉汽车CEO)。他曾在采访中提到自己特别推崇“第一性原理”思考法：“通过第一性原理，我把事情升华到最根本的真理，然后从最核心处开始推理......”
Human: 这位哲学家的最大成就是什么？
AI: 亚里士多德是古希腊哲学家，他在哲学上最大的贡献在于创立了形式逻辑这一重要分支学科。逻辑思维是亚里士多德在众多领域建树卓越的支柱，这种思维方式自始至终贯穿于他的研究、统计和思考之中。

除此之外，他还是一位杰出的科学家，对物理学、生物学、植物学、动物学、解剖学、伦理学等领域都有很大的贡献。
Human: 他在心理学方面的贡献是什么？
AI: 亚里士多德在心理学方面的贡献是他对人类思维和行为的研究。他认为，人的思想和行为是由身体、情感和精神三部分组成的，这三部分相互作用，共同影响着人的行为。

此外，亚里士多德还提出了一种被称为“幸福学”的学科，它研究的是人的幸福和快乐。
Human: 哪里能买到这本书？

> Finished chain.


'您好！您可以在当地的书店或者网上购买亚里士多德的心理学著作。以下是一些可能有用的链接：\n\n- 当当网\n- 京东'

## 示例3 - 图片理解

我们尝试利用Spark API Gateway向星火认知大模型询问图片中的数据。

参考图片：[https://pbs.twimg.com/media/F_c_hrGWcAA3w0p?format=jpg&name=medium](https://pbs.twimg.com/media/F_c_hrGWcAA3w0p?format=jpg&name=medium)

In [20]:
import os

from langchain.chat_models import ChatOpenAI
from langchain.schema.messages import HumanMessage

def summarize(image_url: str, client: ChatOpenAI, prompt = None):
    # gpt4 vision api doc - https://platform.openai.com/docs/guides/vision
    response = client.invoke(
        [
            HumanMessage(
                content=[
                    {
                        "type": "image_url",
                        "image_url": {"url": image_url},
                    },
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            )
        ]
    )
    return response.content

In [4]:
os.environ['OPENAI_API_KEY'] = '您的有效OpenAI API Key'

In [24]:
image_url = "https://pbs.twimg.com/media/F_c_hrGWcAA3w0p?format=jpg&name=medium"
query = "图里的车是什么品牌，什么颜色？"

In [27]:
client = ChatOpenAI(model="gpt-4-vision-preview", max_tokens=1000)
summary = summarize(image_url, client, query)

In [28]:
summary

'图片中的车是宝马（BMW）品牌，颜色是蓝色。'

In [25]:
client = ChatOpenAI(openai_api_base="https://sparkai-gateway.vercel.app/v1/", model="vision", max_tokens=1000)
summary = summarize(image_url, client, query)

In [26]:
summary

'图中的车是宝马，颜色是蓝色。'